In [80]:
import pandas as pd
import numpy as np

from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, f1_score, precision_score,\
recall_score, confusion_matrix, classification_report, accuracy_score 
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.linear_model import SGDClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sys import argv
import gc
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline, Pipeline
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import StandardScaler, Normalizer

In [81]:
path = '/Users/lucia/phd_work/Clpsy/'
features = pd.read_csv(path + 'suicideDetection/features/FreqSentiMotiTopiFea.csv')
#merge with liwc data
liwc = pd.read_csv(path + 'suicideDetection/features/liwcSW.csv')
liwc = liwc.rename(columns = {liwc.columns[2]:'user_id'})
liwcUser = liwc.groupby('user_id').mean().reset_index()
liwcUser = liwcUser.drop(['Source (A)', 'Source (D)'], axis=1)
allfea = pd.merge(features, liwcUser, on = 'user_id', how = 'right')
# merge empath
empath = pd.read_csv(path + 'suicideDetection/features/empathSW.csv')
allfea = pd.merge(allfea, empath, on = 'user_id', how = 'right')
#merge tags
# tags = pd.read_csv(path + 'suicideDetection/features/TagFeaSW.csv')
# allfea = pd.merge(allfea, tags, on = 'user_id', how = 'right')

allfea.shape

(496, 146)

In [75]:
Lmodeltest = pd.read_csv(path + 'data/languageModel/allPtest.csv')
testId = Lmodeltest['user_id']
testId = testId.drop_duplicates(keep='first', inplace=False).to_frame()
X_train = pd.merge(allfea, testId, on = 'user_id', how='right')
y_train = X_train.raw_label 
X_train = X_train.iloc[:, 3:146]
X_train.shape

(248, 143)

In [74]:
#y_train = pd.merge(allfea, testId, on = 'user_id', how='outer')
X_test = allfea[(~allfea.user_id.isin(testId.user_id))]
y_test = X_test.raw_label 
X_test = X_test.iloc[:, 3:146]
X_test.shape


(248, 143)

In [83]:
X = allfea.iloc[:, 3:146]
y = allfea.raw_label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=30)

def SGDclassifier(X_train, X_test, y_train, y_test):
    smote_enn = SMOTEENN(random_state=42)

    clf = make_pipeline(smote_enn, StandardScaler(), SGDClassifier(max_iter= 1000))

    parameters = [{'sgdclassifier__alpha': [0.01, 0.05, 0.001, 0.005], 'sgdclassifier__class_weight':['balanced'],
                  'sgdclassifier__loss': ['hinge','log','modified_huber','squared_hinge', 'perceptron'], 
                   'sgdclassifier__penalty':['none','l1','l2']}]

    grid_search_item = GridSearchCV(clf,
                              param_grid = parameters,
                               scoring = 'accuracy',
                               cv = 5,
                               n_jobs = -1)

    grid_search = grid_search_item.fit(X_train, y_train)

    print('Best scores and best parameters')
    print(grid_search.best_score_)
    print(grid_search.best_params_)

    y_true, y_pred = y_test, grid_search.predict(X_test)
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))

SGDclassifier(X_train, X_test, y_train, y_test)

/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


Best scores and best parameters
0.4207492795389049
{'sgdclassifier__alpha': 0.005, 'sgdclassifier__class_weight': 'balanced', 'sgdclassifier__loss': 'log', 'sgdclassifier__penalty': 'l1'}
              precision    recall  f1-score   support

           1       0.71      0.64      0.67        42
           2       0.13      0.36      0.19        14
           3       0.24      0.42      0.30        24
           4       0.65      0.29      0.40        69

   micro avg       0.42      0.42      0.42       149
   macro avg       0.43      0.43      0.39       149
weighted avg       0.55      0.42      0.44       149

[[27  7  3  5]
 [ 3  5  4  2]
 [ 2  8 10  4]
 [ 6 18 25 20]]


/Users/lucia/anaconda3/lib/python3.6/site-packages/imblearn/pipeline.py:349: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [79]:
Lmodeltest = pd.read_csv(path + 'data/languageModel/allPtest.csv')
testId = Lmodeltest['user_id']
testId = testId.drop_duplicates(keep='first', inplace=False).to_frame()
X_test = pd.merge(allfea, testId, on = 'user_id', how='right')
y_test = X_test.raw_label 
X_test = X_test.iloc[:, 3:146]
X_train = allfea[(~allfea.user_id.isin(testId.user_id))]
y_train = X_train.raw_label 
X_train = X_train.iloc[:, 3:146]
SGDclassifier(X_train, X_test, y_train, y_test)

/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


Best scores and best parameters
0.49193548387096775
{'sgdclassifier__alpha': 0.01, 'sgdclassifier__class_weight': 'balanced', 'sgdclassifier__loss': 'modified_huber', 'sgdclassifier__penalty': 'l1'}
              precision    recall  f1-score   support

           1       0.53      0.51      0.52        76
           2       0.16      0.36      0.23        22
           3       0.32      0.24      0.27        55
           4       0.45      0.40      0.42        95

   micro avg       0.40      0.40      0.40       248
   macro avg       0.36      0.38      0.36       248
weighted avg       0.42      0.40      0.40       248

[[39  8  7 22]
 [ 4  8  3  7]
 [12 13 13 17]
 [19 20 18 38]]


/Users/lucia/anaconda3/lib/python3.6/site-packages/imblearn/pipeline.py:349: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [34]:
stressed = pd.read_csv(path + 'data/clpsych19_training_data/Stressed.csv')
StressedSW = SW.append(stressed, ignore_index=True)
StressedSW.to_csv(path + 'data/clpsych19_training_data/StressedSW.csv')


In [35]:
path = '/Users/lucia/phd_work/Clpsy/'
allP = pd.read_csv(path + 'data/clpsych19_training_data/Btrain_NoNoise.csv')
SW= pd.read_csv(path + 'data/clpsych19_training_data/Btrain_NoNoise_SW.csv')
SW= pd.read_csv(path + 'data/clpsych19_training_data/Btrain_NoNoise_SW.csv')

labels = pd.read_csv(path + 'data/clpsych19_training_data/crowd_train.csv')
labels = labels.loc[labels['user_id'] > 0]
x_train,x_test = train_test_split(labels,test_size=0.5)

#get test and train file
def mergeFile(file, testSave):
    filetrain = pd.merge(file, x_train, on ='user_id', how ='right')
    filetest = pd.merge(file, x_test, on ='user_id', how ='right')
    filetest.to_csv()
    return filetrain

testSave1 = path + 'data/languageModel/allPtest.csv'
mergeFile(allP, testSave1)

testSave2 = path + 'data/languageModel/SWPtest.csv'
mergeFile(SW, testSave2)

#SWPtrain = pd.merge(SW, x_train, on ='user_id', how ='right')
#SWPtest = pd.merge(SW, x_test, on ='user_id', how ='right')
#SWPtest.to_csv()

print(len(allP))
print(len(allPtrain))
print(len(allPtest))

print(len(SW))
print(len(SWPtrain))
print(len(SWPtest))

31553
15728
15825
919
516
403


In [5]:
def getClass(file, Class, savePath):
    a = file.loc[file['raw_label_x'] == Class]
    a = a[['post_body','post_title']]
    a.to_csv(savePath)
    

savePath1 = path + 'data/languageModel/AAll.csv'
savePath2 = path + 'data/languageModel/BAll.csv'
savePath3 = path + 'data/languageModel/CAll.csv'
savePath4 = path + 'data/languageModel/DAll.csv'

ClassA = getClass(allPtrain, 'a', savePath1)
ClassB = getClass(allPtrain, 'b', savePath2)
ClassC = getClass(allPtrain, 'c', savePath3)
ClassD = getClass(allPtrain, 'd', savePath4)

In [7]:
savePath1 = path + 'data/languageModel/ASW.csv'
savePath2 = path + 'data/languageModel/BSW.csv'
savePath3 = path + 'data/languageModel/CSW.csv'
savePath4 = path + 'data/languageModel/DSW.csv'

ClassA = getClass(SWPtrain, 'a', savePath1)
ClassB = getClass(SWPtrain, 'b', savePath2)
ClassC = getClass(SWPtrain, 'c', savePath3)
ClassD = getClass(SWPtrain, 'd', savePath4)

In [147]:
text = pd.read_csv(path + 'data/clpsych19_training_data/Btrain_NoNoise_SW.csv')
countVect = pd.read_csv(path + 'countVec.csv')
countVect['user_id'] = text['user_id']

#text['post_body'] = text[['post_body','user_id']].groupby(['post_body'])['user_id'].transform(lambda x: ','.join(x))

In [149]:
countVect.iloc[:, 3::]

,2,3,4,5,6,7,8,9,10,11,...,7760,7761,7762,7763,7764,7765,7766,7767,7768,user_id
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,22002
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.157057,0.000000,0.0,22006
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.060629,0.000000,0.000000,0.0,22015
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,22134
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,22134
5,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,22134
6,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,22134
7,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,22134
8,0.0,0.0,0.0,0.0,0.128905,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,22134
9,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,22354


In [145]:
countVect.groupby(['user_id']).mean().reset_index()
#text2.to_csv(path+ 'text2.csv')

,user_id,Unnamed: 0,0,1,2,3,4,5,6,7,...,7759,7760,7761,7762,7763,7764,7765,7766,7767,7768
0,16,588.5,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.089694,0.0,0.0,0.0,0.0000,0.0,0.0
1,56,594.5,0.0,0.0,0.0,0.0,0.0,0.0,0.075383,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0
2,137,581.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0
3,152,585.5,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0
4,370,593.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0
5,706,596.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0
6,967,600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023086,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0
7,981,604.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0
8,1215,577.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0
9,1233,578.0,0.0,0.0,0.0,0.0,0.0,0.0,0.115426,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0


In [137]:
for i in text2['post_body']:
    print(i)

0    I was at party. When everyone were having fun....
Name: 22002, dtype: object
1    Hi. So for starters, I take Zoloft, 75g; they ...
Name: 22006, dtype: object
2    I apologize in advance for this being so long....
Name: 22015, dtype: object
3    I made a thread yesterday where I expressed a ...
4                                                  NaN
5                                                  NaN
6    I tried last night, but threw up the pills. I ...
7    ...but I'm pretty depressed. I have intrusive ...
8    That's the name of the poem _PERSON_ wrote as ...
Name: 22134, dtype: object
9     I can't say much but I have transgender FtM fr...
10    I trusted you, ya know. I trusted you with my ...
Name: 22354, dtype: object
11    I know. _URL_ album saved my life one time whe...
12    You don't need to feel alone.I Love You.If you...
13    This has all been inspired by ThoughtlessThink...
14    I'm running a scan on my laptop, so I'll be he...
Name: 22733, dtype: object
15    W

In [134]:
text

,Unnamed: 0,post_id,user_id,timestamp,subreddit,post_title,post_body,raw_label
0,13,2bvvxg,22002,1406494593,SuicideWatch,Yesterday i turned off cigarette in my hand.,I was at party. When everyone were having fun....,a
1,54,20x9u6,22006,1395339794,SuicideWatch,Not sure what to do now.,"Hi. So for starters, I take Zoloft, 75g; they ...",c
2,75,tr7vb,22015,1337230954,SuicideWatch,I despise sounding so pathetically melodramati...,I apologize in advance for this being so long....,d
3,110,ip6fx,22134,1310621370,SuicideWatch,You guys saved me,I made a thread yesterday where I expressed a ...,d
4,114,j2ve3,22134,1311911038,SuicideWatch,Back again. Things got worse. MUCH worse. (x-p...,NaN,d
5,135,xhk4e,22134,1343792229,SuicideWatch,Took _PERSON_ of morphine and some valium,NaN,d
6,136,xjsk1,22134,1343885091,SuicideWatch,Back again.,"I tried last night, but threw up the pills. I ...",d
7,164,1dlole,22134,1367556447,SuicideWatch,Not really suicidal...,...but I'm pretty depressed. I have intrusive ...,d
8,177,1hcjbb,22134,1372564540,SuicideWatch,Football season is over,That's the name of the poem _PERSON_ wrote as ...,d
9,223,2d7dzp,22354,1407730199,SuicideWatch,My friend might be thinking.,I can't say much but I have transgender FtM fr...,a
